In [1]:
# import autograd-wrapped numpy
import autograd.numpy as np
from autograd import grad
from autograd import hessian

# datapath to data
datapath = './superlearn_datasets/'

def model(x, w):
    a = w[0] + np.dot(x.T, w[1:])
    return a.T


def newtons_method(g, max_its, w):
    gradient = grad(g)
    hess = hessian(g)
    epsilon = 1e-7
    
    weight_his = [w]
    cost_his = [g(w)]
    for k in range(max_its):
        
        grad_eval = gradient(w)
        hess_eval = hess(w)
        
        hess_eval.shape = (int(np.size(hess_eval)**0.5),
                           int(np.size(hess_eval)**0.5))
        A = hess_eval + epsilon*np.eye(w.size)
        b = grad_eval
        w = np.linalg.solve(A, np.dot(A,w)-b)
        weight_his.append(w)
        cost_his.append(g(w))
    return weight_his, cost_his

def gradient_descent(g, max_its, w):
    gradient = grad(g)
    weight_his = [w]
    cost_his = [g(w)]
    for k in range(max_its):
        grad_eval = gradient(w)
    
        w = w - grad_eval
        weight_his.append(w)
        cost_his.append(g(w))
    return weight_his, cost_his

# 6.12

In [2]:
csvname = datapath + '2d_classification_data_v1.csv'
data = np.loadtxt(csvname,delimiter=',')
x = data[:-1,:]
y = data[-1:,:]

w = np.ones((2, 1))
l = 1e-3
k = int(1e3)


def softmax(w):
    cost = np.sum(np.log(1+np.exp(-y*model(x,w))))
    return cost/float(np.size(y))


def regular(w):
    return softmax(w) + l*np.dot(w.T, w)

weight_his, cost_his = newtons_method(regular, k, w)
mis_classified = np.sum(np.apply_along_axis(lambda x : 1 if x < 0 else 0, 0, (y*model(x, weight_his[-1]).squeeze())))
print(mis_classified)

0


# 6.13

In [3]:
csvname = datapath + 'breast_cancer_data.csv'
data = np.loadtxt(csvname,delimiter = ',')

# get input and output of dataset
x = data[:-1,:]
y = data[-1:,:]

def perceptron(w):
    cost = np.sum(-y*model(x, w) * (-y*model(x, w) > 0))
    return cost/float(np.size(y))


N = 9
m1 ,m2 = 100, 100
for _ in range(100):
    w = np.random.rand(N,1)
    weight_his, cost_his = gradient_descent(softmax, k, w)
    mis_classified = np.sum(np.apply_along_axis(lambda x : 1 if x < 0 else 0, 0, (y*model(x, weight_his[-1]).squeeze())))
    m1 = min(m1, mis_classified)



    w = np.random.rand(N,1)
    weight_his, cost_his = gradient_descent(perceptron, k, w)
    mis_classified = np.sum(np.apply_along_axis(lambda x : 1 if x < 0 else 0, 0, (y*model(x, weight_his[-1]).squeeze())))
    m2 = min(m2, mis_classified)
print(m1, m2)

21 21


We can find that they are almost the same.

# 6.15

In [4]:
# load in dataset
csvname = datapath + 'credit_dataset.csv'
data = np.loadtxt(csvname,delimiter = ',')
x = data[:-1,:]
y = data[-1:,:] 

def norm(x):
    mu = np.sum(np.array(x), 1) / float(x.shape[1])
    tmp = np.apply_along_axis(lambda x : x - mu, 0, x)
    sigma = np.sqrt(np.sum(np.apply_along_axis(lambda x : x - mu, 0, x)**2, 1) / float(x.shape[1]))
    x = np.apply_along_axis(lambda x : (x - mu) / sigma, 0, x)
    return x

x = norm(x)
N = 21
w = np.random.rand(N, 1)

weight_his, cost_his = gradient_descent(softmax, k, w)
A, B, C, D = 0, 0, 0, 0
for t, p in zip(y.squeeze(), model(x, weight_his[-1]).squeeze()):
    if (p < 0 and t < 0):
        A += 1
    elif (p > 0 and t < 0):
        B += 1
    elif (p < 0 and t > 0):
        C += 1
    else:
        D += 1
print(np.array([[A, B], [C, D]]))
print(float(A + D) / float(C + B + D + A))

[[144 156]
 [ 77 623]]
0.767


# 6.16

In [5]:
# data input
csvname = datapath + '3d_classification_data_v2_mbalanced.csv'
data1 = np.loadtxt(csvname,delimiter = ',')

# get input and output of dataset
x = data1[:-1,:]
y = data1[-1:,:]

def weighted_softmax(w):
    ind = np.argwhere(y > 0)[:,1]
    cost = np.sum(beta*np.log(1+np.exp(-y*model(x,w)))[:, ind])
    ind = np.argwhere(y < 0)[:,1]
    cost += np.sum(np.log(1+np.exp(-y*model(x,w)))[:, ind])
    return cost/float(np.size(y))

k = 5

beta = 1
w = np.random.rand(3,1)

weight_his, cost_his = newtons_method(weighted_softmax, k, w)
#print(weight_his[-1])
A, B, C, D = 0, 0, 0, 0
for t, p in zip(y.squeeze(), model(x, weight_his[-1]).squeeze()):
    if (p < 0 and t < 0):
        A += 1
    elif (p > 0 and t < 0):
        B += 1
    elif (p < 0 and t > 0):
        C += 1
    else:
        D += 1

A1 = float(A) / float(A + B)
A2 = float(D) / float(C + D)
accuracy = (A1 + A2) / 2
print(beta, accuracy)

beta = 5
w = np.random.rand(3,1)

weight_his, cost_his = newtons_method(weighted_softmax, k, w)
#print(weight_his[-1])
A, B, C, D = 0, 0, 0, 0
for t, p in zip(y.squeeze(), model(x, weight_his[-1]).squeeze()):
    if (p < 0 and t < 0):
        A += 1
    elif (p > 0 and t < 0):
        B += 1
    elif (p < 0 and t > 0):
        C += 1
    else:
        D += 1
A1 = float(A) / float(A + B)
A2 = float(D) / float(C + D)
accuracy = (A1 + A2) / 2
print(beta, accuracy)


beta = 10
w = np.random.rand(3,1)

weight_his, cost_his = newtons_method(weighted_softmax, k, w)
#print(weight_his[-1])
A, B, C, D = 0, 0, 0, 0
for t, p in zip(y.squeeze(), model(x, weight_his[-1]).squeeze()):
    if (p < 0 and t < 0):
        A += 1
    elif (p > 0 and t < 0):
        B += 1
    elif (p < 0 and t > 0):
        C += 1
    else:
        D += 1
A1 = float(A) / float(A + B)
A2 = float(D) / float(C + D)
accuracy = (A1 + A2) / 2
print(beta, accuracy)

1 0.79
5 0.87
10 0.96
